<a href="https://colab.research.google.com/github/Alpha201488/Open-AutoGLM/blob/main/Z_Image_Turbo_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/comfyanonymous/ComfyUI

%cd /content/ComfyUI
!pip install -r requirements.txt

!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d /content/ComfyUI/models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors -d /content/ComfyUI/models/clip -o qwen_3_4b.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors -d /content/ComfyUI/models/vae -o ae.safetensors

fatal: destination path 'ComfyUI' already exists and is not an empty directory.
/content/ComfyUI
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
b3a0f7|OK  |       0B/s|/content/ComfyUI/models/diffusion_models/z-image-turbo-fp8-e4m3fn.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
1a7a7c|OK  |       0B/s|/content/ComfyUI/models/clip/qwen_3_4b.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
ab5caf|OK  |       0B/s|/content/ComfyUI/models/vae/ae.safetensors

Status Legend:
(OK):download completed.


In [5]:
%cd /content/ComfyUI

import os, random, time
import torch
import numpy as np
from PIL import Image

from nodes import NODE_CLASS_MAPPINGS

UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
CLIPLoader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet = UNETLoader.load_unet("z-image-turbo-fp8-e4m3fn.safetensors", "fp8_e4m3fn_fast")[0]
    clip = CLIPLoader.load_clip("qwen_3_4b.safetensors", type="lumina2")[0]
    vae = VAELoader.load_vae("ae.safetensors")[0]

# ========= callback =========
def step_callback(step, x0, x, total_steps):
    global _preview_vae, _preview_outdir
    if step >= total_steps - 1:
        return

    decoded = VAEDecode.decode(_preview_vae, x)[0].detach()
    img = Image.fromarray(
        (decoded[0].clamp(0,1).cpu().numpy() * 255).astype(np.uint8)
    )
    img.save(f"{_preview_outdir}/step_{step:02d}.png")


@torch.inference_mode()
def generate(input):
    tmp_dir="/content/ComfyUI/output"
    os.makedirs(tmp_dir, exist_ok=True)

    values = input["input"]

    positive_prompt = values['positive_prompt']
    negative_prompt = values['negative_prompt']
    seed = values['seed']
    steps = values['steps']
    cfg = values['cfg']
    sampler_name = values['sampler_name']
    scheduler = values['scheduler']
    denoise = values['denoise']
    width = values['width']
    height = values['height']
    batch_size = values['batch_size']

    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)

    positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    negative = CLIPTextEncode.encode(clip, negative_prompt)[0]
    latent_image = EmptyLatentImage.generate(width, height, batch_size=batch_size)[0]

    """global _preview_vae, _preview_outdir
    _preview_vae = vae
    _preview_outdir = tmp_dir"""

    global _preview_vae, _preview_outdir
    _preview_vae = vae
    _preview_outdir = tmp_dir

    noise = torch.randn(latent_image.shape, device=unet.load_device)

    real_sampler = _RealKSampler(
        unet,
        steps,
        unet.load_device,
        sampler=sampler_name,
        scheduler=scheduler,
        denoise=denoise
    )

    samples = real_sampler.sample(
        noise,
        positive,
        negative,
        cfg,
        latent_image=latent_image,
        callback=step_callback,
        seed=seed
    )


    decoded = VAEDecode.decode(vae, samples)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(f"{tmp_dir}/z_image_turbo.png")

    return f"{tmp_dir}/z_image_turbo.png"


/content/ComfyUI


In [2]:
input = {
  "input": {
    "positive_prompt": "一位男士和他的贵宾犬穿着配套的服装参加狗狗秀，室内灯光，背景中有观众。",
    "negative_prompt": "blurry ugly bad",
    "width": 1024,
    "height": 1024,
    "batch_size": 1,
    "seed": 0,
    "steps": 9,
    "cfg": 1,
    "sampler_name": "euler",
    "scheduler": "simple",
    "denoise": 1.0,
  }
}

output = generate(input)
Image.open(output)

TypeError: KSampler.sample() got an unexpected keyword argument 'callback'